In [5]:
import pandas as pd
import re
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from operator import itemgetter
from typing import List, Optional, Dict

In [6]:
def extract_test_codes(test_types_str: str) -> List[str]:
    """Convert test type string into list of codes"""
    return re.findall(r'[A-Z]', test_types_str)

def create_test_checker(target_code: str):
    """Factory function to create test type validators"""
    def checker(test_codes: List[str]) -> bool:
        return target_code in test_codes
    return checker

In [7]:
test_type_mapping = {
    'A': 'Ability & Aptitude',
    'B': 'Biodata & Situational Judgement',
    'C': 'Competencies',
    'D': 'Development & 360',
    'E': 'Assessment Exercises',
    'K': 'Knowledge & Skills',
    'P': 'Personality & Behavior',
    'S': 'Simulations'
}

In [8]:
def prepare_assessment_data(records: List, category: str) -> pd.DataFrame:
    """Structure assessment data with proper type checking"""
    df = pd.DataFrame(records, columns=[
        'Assessment Name',
        'Remote Testing',
        'Adaptive/IRT',
        'Test Types'
    ])

    # Process test types
    df['Test Types List'] = df['Test Types'].apply(extract_test_codes)

    # Create boolean columns for each test type
    for code, type_name in test_type_mapping.items():
        check_function = create_test_checker(code)
        df[type_name] = df['Test Types List'].apply(check_function)

    df['Category'] = category
    return df.astype({
        'Remote Testing': 'bool',
        'Adaptive/IRT': 'bool',
        'Category': 'category'
    })

In [9]:
class AssessmentRecommender:
    def __init__(self, assessment_data: pd.DataFrame):
        self.data = assessment_data
        self._prepare_keyword_index()

    def _prepare_keyword_index(self):
        """Create quick-access index for job title keywords"""
        self.keyword_index = {}
        for name in self.data['Assessment Name']:
            clean_name = name.replace(' Solution', '').lower()
            self.keyword_index[name] = set(re.findall(r'\w+', clean_name))

    def _calculate_title_similarity(self, target: str, candidate: str) -> float:
        """Determine how well job title matches assessment name"""
        target_words = set(re.findall(r'\w+', target.lower()))
        candidate_words = self.keyword_index.get(candidate, set())

        if not target_words:
            return 0.0

        common_words = target_words & candidate_words
        return len(common_words) / len(target_words)

    def find_recommendations(self,
                            job_title: Optional[str] = None,
                            required_skills: Optional[List[str]] = None,
                            remote_only: bool = False,
                            max_results: int = 5) -> pd.DataFrame:
        """Main recommendation generator with skill prioritization"""

        filtered = self.data.copy()

        # Apply basic filters
        if remote_only:
            filtered = filtered[filtered['Remote Testing']]

        if required_skills:
            skill_columns = [test_type_mapping[s] for s in required_skills]
            filtered = filtered[filtered[skill_columns].all(axis=1)]

        # Score and rank results
        results = []
        for _, assessment in filtered.iterrows():
            # Calculate relevance scores
            title_score = self._calculate_title_similarity(
                job_title or '',
                assessment['Assessment Name']
            ) if job_title else 0

            skill_score = 1.0  # Default if no skills specified
            if required_skills:
                matched = sum(assessment[test_type_mapping[s]] for s in required_skills)
                skill_score = matched / len(required_skills)

            # Combine scores with priority to title matches
            combined_score = (0.6 * title_score) + (0.4 * skill_score)

            results.append((assessment, combined_score))

        # Sort and return best matches
        results.sort(key=itemgetter(1), reverse=True)
        best_matches = [item[0] for item in results[:max_results]]

        return pd.DataFrame(best_matches).assign(
            Match_Score=[item[1] for item in results[:max_results]]
        )

In [10]:
class RecommendationInterface:
    def __init__(self, recommender: AssessmentRecommender):
        self.engine = recommender
        self._setup_interface()

    def _setup_interface(self):
        """Build interactive components with clear organization"""
        self.job_input = widgets.Text(
            placeholder='e.g., "Bank Manager"',
            description='Job Role:'
        )

        self.skill_selectors = {
            code: widgets.Checkbox(description=f"{code} - {name}")
            for code, name in test_type_mapping.items()
        }

        self.remote_toggle = widgets.Checkbox(
            value=True,
            description='Require Remote Testing'
        )

        self.results_slider = widgets.IntSlider(
            value=5,
            min=1,
            max=15,
            description='Results to Show:'
        )

        self.search_button = widgets.Button(
            description='Find Assessments',
            button_style='primary'
        )
        self.search_button.on_click(self._handle_search)

        self.output_area = widgets.Output()

        # Assemble interface layout
        self.interface = widgets.VBox([
            widgets.HTML("<h1 style='color: #1f77b4;'>SHL Assessment Advisor</h1>"),
            self.job_input,
            widgets.HTML("<h3>Essential Skills:</h3>"),
            widgets.GridBox(
                list(self.skill_selectors.values()),
                layout=widgets.Layout(grid_template_columns="repeat(3, 200px)")
            ),
            widgets.HBox([self.remote_toggle, self.results_slider]),
            self.search_button,
            self.output_area
        ])

    def _handle_search(self, button):
        """Process user input and display results"""
        with self.output_area:
            clear_output()

            # Collect user preferences
            selected_skills = [
                code for code, widget in self.skill_selectors.items()
                if widget.value
            ]

            try:
                recommendations = self.engine.find_recommendations(
                    job_title=self.job_input.value,
                    required_skills=selected_skills,
                    remote_only=self.remote_toggle.value,
                    max_results=self.results_slider.value
                )

                if not recommendations.empty:
                    # Format results display
                    styled_results = recommendations.style.format({
                        'Match_Score': '{:.1%}'
                    }).bar(subset=['Match_Score'], color='#5c90d2')

                    display(HTML("<h3>Recommended Assessments</h3>"))
                    display(styled_results)
                    self._display_top_assessment(recommendations.iloc[0])
                else:
                    display(HTML("<p style='color: #666;'>No matching assessments found. Try broader criteria.</p>"))

            except Exception as error:
                display(HTML(f"<p style='color: #cc0000;'>Error: {str(error)}</p>"))

    def _display_top_assessment(self, assessment):
        """Show detailed view for top recommendation"""
        details = widgets.Accordion(titles=['Assessment Details'])

        content = widgets.HTML(f"""
            <div style='background: #f8f9fa; padding: 15px; border-radius: 5px;'>
                <h3>{assessment['Assessment Name']}</h3>
                <p><b>Category:</b> {assessment.Category}</p>
                <p><b>Remote Available:</b> {'Yes' if assessment['Remote Testing'] else 'No'}</p>
                <p><b>Test Types:</b> {assessment['Test Types']}</p>
                <p><b>Adaptive Format:</b> {'Yes' if assessment['Adaptive/IRT'] else 'No'}</p>
            </div>
        """)

        details.children = [content]
        display(details)

In [11]:
def initialize_system():
    """Configure and launch the recommendation system"""
    # Load and prepare data
    job_solutions = prepare_assessment_data(job_solutions_data, 'Job-Specific Packages')
    individual_tests = prepare_assessment_data(individual_tests_data, 'Skill Assessments')
    full_dataset = pd.concat([job_solutions, individual_tests], ignore_index=True)

    # Create and display interface
    recommender = AssessmentRecommender(full_dataset)
    interface = RecommendationInterface(recommender)
    display(interface.interface)

    # Add styling for better visual presentation
    display(HTML("""
    <style>
        .widget-label { min-width: 120px !important; }
        .widget-slider { width: 300px; }
        .widget-checkbox { margin: 5px 15px; }
    </style>
    """))

In [12]:
initialize_system()